# World Health Organization Covid-19 Reporting

In [1]:
from bs4 import BeautifulSoup
import requests, wget, datetime, PyPDF2

In [14]:
site = "https://www.who.int/emergencies/diseases/novel-coronavirus-2019/situation-reports"
req = requests.get(site)
soup = BeautifulSoup(req.content, 'html.parser')

In [37]:
sitrep = 'situation-reports' + '_' + str(datetime.date.today())
with open(sitrep, 'w') as f:
    f.writelines(req.text)

In [124]:
reports = [ 'https://www.who.int' + link for link in \
          [ l.get('href') for l in soup.find_all('a') ] \
          if link.startswith('/docs') ]

In [132]:
def dl_todays_report():
    wget.download(reports[1])

def dl_all_reports():
    for r in reports:
        wget.download(r)

#verify all reports dl'd and not tmp'd or dup'd
#if reports all dl'd but for today, only get today

In [1]:
#need programatic selection of filename
rr_file = open('20200331-sitrep-71-covid-19.pdf','rb')

In [4]:
rr = PyPDF2.PdfFileReader(rr_file)

In [5]:
page = ''
for p in range(rr.getNumPages()):
    page += rr.getPage(p).extractText()

SURVEILLANCE

Date

Area | Total Cases | New Cases | Total Deaths | New Deaths | Since Last Reported Case


In [222]:
#pull SURVEILLANCE BLOCK
#begins with 'SURVEILLANCE'
#ends with |^GRAND TOTAL| after \n 


# pull date
cases

and deaths

. Data as of

In [77]:
#pull date
def get_date():
    dotData=[ page.splitlines().index(l) for l in page.splitlines() if l.startswith('. Data') ][0]
    day = page.splitlines()[dotData+2]
    month = page.splitlines()[dotData+4]
    year = page.splitlines()[dotData+6]
    date = "%s-%s-%s" % (month,day,year)
    return date

# pull table data

In [213]:
#pull table data
report_begin=[ page.splitlines().index(l) for l in page.splitlines() if l.startswith('reported case') ][0] +2
report_end=[ page.splitlines().index(l) for l in page.splitlines() if l.startswith('Numbers') ][0] 
#surveillance_data = [ c.strip('\"') for c in \
#                    [ l for l in page.splitlines()[report_begin:report_end] if l != ' ' ] ]
surveillance_data = [ l for l in page.splitlines()[report_begin:report_end] if l != ' ' ]


In [ ]:
#if 'REGION' in line, line = region
#if 'Local transmission' in line, line[+2] = loc_trans 
#  and line[+3] = end

In [191]:
def get_data():
    result = {}
    for l in surveillance_data:
        region = ''
        area = ''
        if 'Region' in l:
            region = l
        else:
            region = region
            for c in l:
                if c.isalpha() or c.isspace():
                    area += c
            if area != '':
                block_begin = surveillance_data.index(area)
                total_confirmed = surveillance_data[block_begin+1]
                total_new = surveillance_data[block_begin+2]
                total_deaths = surveillance_data[block_begin+3]
                total_new_deaths = surveillance_data[block_begin+4]
                result[region] =  [ area,total_confirmed,total_new,total_deaths,total_new_deaths]
    return result

In [219]:
for l in surveillance_data:
    region = ''
    area = ''
    trans = ''
    if 'Region' in l:
        region = l
    else if region
        region = region
        if 'Imported' in l or 'Local' in l:
            trans = l
        else:
            for c in l:
                if c.isalpha() or c.isspace():
                    area += c           
        if area:
            block_begin = surveillance_data.index(area)
            total_confirmed = surveillance_data[block_begin+1]
            total_new = surveillance_data[block_begin+2]
            total_deaths = surveillance_data[block_begin+3]
            total_new_deaths = surveillance_data[block_begin+4]
            print(region,[ area,total_confirmed,total_new,total_deaths,total_new_deaths])

 ['China', '82545', '98', '3314', '4']
 ['Republic of Korea', '9786', '125', '162', '4']
 ['Australia', '4359', '393', '18', '2']
 ['Malaysia', '2626', '156', '37', '3']
 ['Japan', '1953', '87', '56', '2']
 ['Philippines', '1546', '128', '78', '7']
 ['Singapore', '879', '35', '3', '0']
 ['New Zealand', '600', '48', '1', '0']
 ['Viet Nam', '203', '15', '0', '0']
 ['Brunei Darussalam', '127', '1', '1', '0']
 ['Cambodia', '107', '4', '0', '0']
 ['Mongolia', '12', '0', '0', '0']
 ['Imported cases only', '3', "Lao People's ", 'Democratic Republic', '8']


ValueError: 'Lao Peoples ' is not in list

In [237]:
region_index = []
for ri in surveillance_data:
    if 'Region' in ri:
        #region_index[ri] = surveillance_data.index(ri)
        region_index.append(surveillance_data.index(ri))
region_index

[0, 140, 572, 645, 798, 1167]

In [273]:
ai = {}
for i in range(len(region_index)):
    for idx in range(i,region_index[i-1]):
        ai[surveillance_data[region_index[i]]] = [ idx , ]

In [274]:
ai

{'Western Pacific Region': [1166],
 'East Asia Region': [139],
 'Eastern Mediterranean Region': [571],
 'Region of the Americas': [644],
 'African Region': [797]}

In [268]:
region_index[5]

1167

In [238]:
surveillance_data[140]

'European Region '